In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [27]:
chrome_driver_path = '/Users/scottheng/Projects/linkedin-job-analysis/chromedriver'

sf_swe_jobs_url = 'https://www.linkedin.com/jobs/software-engineering-jobs-san-francisco-bay-area/?geoId=90000084'

# remove this
linkedin_username = ''
linkedin_password = ''

In [28]:
# initialize driver
driver = webdriver.Chrome(chrome_driver_path)

# connect to linkedin
driver.get('https://www.linkedin.com')

In [29]:
elementID = driver.find_element_by_id('session_key')
elementID.send_keys(linkedin_username)
elementID = driver.find_element_by_id('session_password')
elementID.send_keys(linkedin_password)

elementID.submit()

In [30]:
driver.get(sf_swe_jobs_url)

In [32]:
# get all jobIds
jobSearchResults = driver.find_elements_by_class_name('jobs-search-results__list-item')
jobIdList = []

for result in jobSearchResults:
    attribute = result.get_attribute('data-occludable-entity-urn')
    attribute = attribute.split(':')
    jobIdList.append(attribute[-1])

In [33]:
# make jobId URLs
linkedin_url_prefix = 'https://www.linkedin.com/jobs/view/'
jobIdUrls = []

for jobId in jobIdList:
    url = linkedin_url_prefix + jobId
    jobIdUrls.append(url)

- Find jobs across multiple pages (pagination w. Spark)
- Make sure search_url is updated

In [34]:
attributeMap = {
    'job_title': ('class_name','t-24'),
    'company_name': ('id','ember47'),
    'job_location': ('class_name','jobs-unified-top-card__bullet'),
    'posted_date_relative': ('class_name','jobs-unified-top-card__posted-date'),
    'applicant_count': ('class_name','jobs-unified-top-card__applicant-count'),
    'job_insight': ('class_name','jobs-unified-top-card__job-insight'),
    'job_details': ('id','job-details')
}

# gets the attribute web element (after url has been visited) and returns text or empty
def getWebElementTextFromDriver(driver, attribute):
    attribute_mapping = attributeMap[attribute]
    try:
        if attribute_mapping[0] == 'class_name':
            webElement = driver.find_element_by_class_name(attribute_mapping[1])
        elif attribute_mapping[0] == 'id':
            webElement = driver.find_element_by_id(attribute_mapping[1])
        return webElement.text
    
    except:
        return ''

def parseLinkedInData(driver, url, attributeMap):
    driver.get(url)
    data = {}
    
    for attribute in attributeMap:
        data[attribute] = getWebElementTextFromDriver(driver,attribute)
        
    return data

In [36]:
# get info from job url
index = 0
dataList = []

for url in jobIdUrls:
    jobData = parseLinkedInData(driver, url, attributeMap)
    dataList.append(jobData)

In [39]:
# create pandas dataframe

# initialize
df = pd.DataFrame()

for entry in dataList:
    df = df.append(entry, ignore_index=True)

In [41]:
# save to csv
csv_file = df.to_csv('linkedin_data')